# Module 1 - Calculating seasonal maps and resampling WaPOR climate data (RET & PCP)
This Notebook contains the following steps
* Step a - Import modules/libraries
* Step b - Calculate seasonal PCP and RET data
* Step c - Resampling seasonal PCP and RET raster data

**=====================================================================================================================**

![image](https://raw.githubusercontent.com/wateraccounting/WaPORIPA/main/images/Resampling.png)

**=====================================================================================================================**

Before you start running this Notebook, make sure you have the following information:

*   WaPOR Reference ET and PCP for your case study for the period you want to do the analyses. Use this script for downloading the data: [Downloading_WaPORv3_data script](https://github.com/wateraccounting/WaPORMOOC/tree/main/1_WaPOR_download_colab). *For small areas make sure you download the data using a bounding box to avoid non data*

*   Sample of a TIFF file with the resolution you want to do the analyses

## Step a - Import modules/libraries

In [ ]:
!pip install --upgrade rioxarray --quiet

In [2]:
import os                             # a module for interacting with the operating system
import glob                           # used to retrieve files/pathnames matching a specified pattern
import matplotlib.pyplot as plt       # is a plotting library used for 2D graphics in python
import numpy as np                    # stands for 'Numerical Python, is a python library used for scientific computing with arrays
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as rioxr
import rasterio as rio
import pandas as pd                       # to store and manipulate tabular data in rows of observations and columns of variables
import calendar
import datetime

from osgeo import ogr, gdal
import subprocess


To import some useful GISfunctions, zip the Modules folder from github and upload it here.

In [3]:
#To upload modules.zip (importing GISfunctions to do the calculations).
from google.colab import files
uploaded = files.upload()

Saving Modules.zip to Modules.zip


In [4]:
# unzip the uploaded zipfile
!unzip '/content/Modules.zip' -d '/content/'

Archive:  /content/Modules.zip
   creating: /content/Modules/
   creating: /content/Modules/GIS_functions/
  inflating: /content/Modules/GIS_functions/GIS_function.py  
 extracting: /content/Modules/GIS_functions/__init__.py  
   creating: /content/Modules/GIS_functions/__pycache__/
  inflating: /content/Modules/GIS_functions/__pycache__/GIS_function.cpython-37.pyc  
  inflating: /content/Modules/GIS_functions/__pycache__/__init__.cpython-37.pyc  
   creating: /content/Modules/WaPOR/
  inflating: /content/Modules/WaPOR/GIS_functions.py  
  inflating: /content/Modules/WaPOR/WaitbarConsole.py  
  inflating: /content/Modules/WaPOR/WaporAPI.py  
  inflating: /content/Modules/WaPOR/__init__.py  
  inflating: /content/Modules/WaPOR/__init__.pyc  
   creating: /content/Modules/WaPOR/__pycache__/
  inflating: /content/Modules/WaPOR/__pycache__/AET_dekadal.cpython-36.pyc  
  inflating: /content/Modules/WaPOR/__pycache__/AET_dekadal.cpython-37.pyc  
  inflating: /content/Modules/WaPOR/__pycache_

In [5]:
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "/content/Modules"))
from GIS_functions import GIS_function as gis

Changing directory back to main directory

In [6]:
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "/content"))

## Import the input data and sample of resolution

##Import downloaded RET and PCP files

Import WaPOR RET and PCP using [Downloading_WaPORv3_data script](https://github.com/wateraccounting/WaPORMOOC/tree/main/1_WaPOR_download_colab).

*Download using bounding box (bb) not shapefile to avoid non data.*

In [ ]:
#To upload data.zip.
from google.colab import files
uploaded = files.upload()

In [9]:
# unzip the uploaded zipfile
!unzip '/content/data.zip' -d '/content/'

Archive:  /content/data.zip
   creating: /content/content/output/
   creating: /content/content/output/L1-RET-D/
  inflating: /content/content/output/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2022-10-01.tif  
  inflating: /content/content/output/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-03-01.tif  
  inflating: /content/content/output/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-01-21.tif  
  inflating: /content/content/output/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2022-12-01.tif  
  inflating: /content/content/output/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2022-12-21.tif  
  inflating: /content/content/output/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2022-11-01.tif  
  inflating: /content/content/output/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-01-11.tif  
  inflating: /content/content/output/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-03-21.tif  
  inflating: /content/content/output/L1-RET-D/bb_L1-RET-D_NONE_dekad_converted_2023-04-11.tif  
  inflating: /content/c

# Step b - Create seasonal climate layers


Create input and output folders

In [37]:
target_folderPCP = '/content/content/output/L1-PCP-D'                       # location of data to be aggregated
target_fhsPCP    = target_fhsPCP
output_folderPCP_season = "/content/output_seasonal/PCP_Season"

if not os.path.exists(output_folderPCP_season):
    os.makedirs(output_folderPCP_season)

target_folderRET = '/content/content/output/L1-RET-D'                      # location of data to be aggregated
target_fhsRET    = glob.glob(target_folderRET + '/*.tif')

output_folderRET_season = "/content/output_seasonal/RET_season"

if not os.path.exists(output_folderRET_season):
    os.makedirs(output_folderRET_season)

output_folderPCP_season, output_folderRET_season

('/content/output_seasonal/PCP_Season', '/content/output_seasonal/RET_season')

Define seasons

In [36]:
season_periods = {
    'season1': {'SOS': '2022-10-01', 'EOS': '2023-04-30'}
}

###Define function for temporal aggregation

This script defines the function `SumSeason `which is used in the next sections. We need this function to aggregate PCP and RET data.


In [14]:
# summation of raster between two dates
def SumSeason(input_fhs, sowing_date, harvesting_date):

    # add rasters that falls within sowing and harvesting date
    Sums = 0
    for i, in_fh in enumerate(input_fhs):
        # open raster file and get start and end date from attributes
        ds = rioxr.open_rasterio(in_fh)
        ds = ds.where(ds!=ds.attrs['_FillValue'])
        if i == 0:
          attrs = ds.attrs
        raster_startdate = datetime.datetime.strptime(ds.attrs['start_date'], '%Y-%m-%d' ).date()
        raster_enddate   = datetime.datetime.strptime(ds.attrs['end_date'], '%Y-%m-%d' ).date()

        # accumulate full or part of the dekadal value
        if (raster_startdate >= sowing_date) & (raster_enddate <= harvesting_date):
            Sum = ds
            Sums += Sum
#             print ("1",in_fh)

        elif (raster_startdate <sowing_date)&(raster_enddate >sowing_date)&(raster_enddate <harvesting_date):
            Sum = ds
            Sum = Sum*((raster_enddate-sowing_date)/(raster_enddate-raster_startdate))
            Sums += Sum
#             print ("2",in_fh)

        elif (raster_startdate >sowing_date)&(raster_startdate <harvesting_date)&(raster_enddate >harvesting_date):
            Sum = ds
            Sum = Sum*((harvesting_date- raster_startdate)/(raster_enddate-raster_startdate))
            Sums += Sum
#             print ("3",in_fh)

        elif (sowing_date>=raster_startdate)&(harvesting_date<=raster_enddate):
            Sum = ds
            Sum = Sum*((harvesting_date- sowing_date)/(raster_enddate-raster_startdate))
            Sums += Sum
#             print ("4",in_fh)
    attrs.update({'start_date':datetime.datetime.strftime(SOS, '%Y-%m-%d'),
              'end_date': datetime.datetime.strftime(EOS, '%Y-%m-%d'),
              'units' : 'mm/season'})
    del attrs['number_of_days']
    del attrs['temporal_resolution']
    del attrs['units_conversion_factor']

    Sums.attrs  = attrs
    return Sums

Calculate seasonal **PCP**

In [ ]:
# Assign inputs to the SumSeason function
input_fhs = target_fhsPCP
output_folder  = output_folderPCP_season


for i, season in enumerate(season_periods):
    # calculate the seasonal value and save in output_folder
    SOS = datetime.datetime.strptime(season_periods[season]['SOS'],'%Y-%m-%d').date()
    EOS = datetime.datetime.strptime(season_periods[season]['EOS'],'%Y-%m-%d').date()
    seasonal = SumSeason(input_fhs, SOS, EOS)

    ## save the array in raster format, name it with the raster_id and sowing and harvesting date
    out_fh = os.path.join(output_folder, 'PCP' + season + '_' + season_periods[season]['SOS']+'_to_'+season_periods[season]['EOS']+ '.tif')
    seasonal.rio.to_raster(out_fh)


Calculate seasonal **Reference ET**

In [ ]:
# Assign inputs to the SumSeason function
input_fhs      = target_fhsRET
output_folder  = output_folderRET_season

for i, season in enumerate(season_periods):
    # calculate the seasonal value and save in output_folder
    SOS = datetime.datetime.strptime(season_periods[season]['SOS'],'%Y-%m-%d').date()
    EOS = datetime.datetime.strptime(season_periods[season]['EOS'],'%Y-%m-%d').date()
    seasonal = SumSeason(input_fhs, SOS, EOS)

    ## save the array in raster format, name it with the raster_id and sowing and harvesting date
    out_fh = os.path.join(output_folder, 'RET' + season + '_' + season_periods[season]['SOS']+'_to_'+season_periods[season]['EOS']+ '.tif')
    seasonal.rio.to_raster(out_fh)



# Step c - Resample seasonal raster data



The data needs to be resampled to the same resolution as the NPP and AETI data to be able to use in the next module. You need an example file (TIFF file) with this resolution at hand.

Import the example file with the required resolution. 

In [10]:
#To upload sample resolution file.
from google.colab import files
uploaded = files.upload()

Saving AETIseason1_2022-10-01_to_2023-04-30.tif to AETIseason1_2022-10-01_to_2023-04-30.tif


## i) Resample Reference ET

In [78]:

source_file   = '/content/AETIseason1_2022-10-01_to_2023-04-30.tif'   # Read gdal info of template raster file
target_folder = '/content/output_seasonal/RET_season'                 # folder of data to be resampled
target_fhsRET_season =  glob.glob(target_folder + '/*.tif')

# create output folder
output_folder = '/content/content/resampled/L1-RET-D_resampled'

## Make one if the folder does not exit
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

source_file, target_folder, target_fhsRET_season, output_folder

('/content/AETIseason1_2022-10-01_to_2023-04-30.tif',
 '/content/output_seasonal/RET_season',
 ['/content/output_seasonal/RET_season/RETseason1_2022-10-01_to_2023-04-30.tif'])

Check the size and shape of the raster files

In [79]:
## The size and shape of the raster files
template   = gis.OpenAsArray(source_file, nan_values=True)
original   = gis.OpenAsArray(target_fhsRET_season[0], nan_values=True)

print ('The size & shape of the template raster      =', template.size,  '&', template.shape)
print ('The size & shape of the data to be resampled =', original.size,  '&', original.shape)

The size & shape of the template raster      = 353694 & (466, 759)
The size & shape of the data to be resampled = 2 & (1, 2)


Resample raster data

In [81]:
Resample = gis.MatchProjResNDV (source_file, target_fhsRET_season, output_folder, resample = 'near', dtype = 'float32')

Check the size and shape of the raster files

In [83]:
## Check the size and shape of the resampled raster files
Resampled   = "/content/content/resampled/L1-RET-D_resampled/RETseason1_2022-10-01_to_2023-04-30.tif"
resampled   = gis.OpenAsArray(Resampled , nan_values=True)

print ('The size & shape of the resampled data =', resampled.size,  '&', resampled.shape)

The size & shape of the resampled data = 353694 & (466, 759)


## ii) Resample Precipitation layers

In [65]:
source_file   = '/content/AETIseason1_2022-10-01_to_2023-04-30.tif'     # Read gdal info of template raster file
target_folder = '/content/output_seasonal/PCP_Season'                      # data to be resampled
target_fhsPCP    = glob.glob(target_folder + '/*.tif')

# Create output folder
output_folder = '/content/content/resampled/L1-PCP-D_resampled'

## Make one if the folder does not exit
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


source_file, target_folder, target_fhsPCP, output_folder

('/content/AETIseason1_2022-10-01_to_2023-04-30.tif',
 '/content/output_seasonal/PCP_Season',
 ['/content/output_seasonal/PCP_Season/PCPseason1_2022-10-01_to_2023-04-30.tif'])

Check the size and shape of the raster files

In [68]:
## The size and shape of the raster files
template   = gis.OpenAsArray(source_file, nan_values=True)
original   = gis.OpenAsArray(target_fhsPCP[0], nan_values=True)

print ('The size & shape of the template raster      =', template.size,  '&', template.shape)
print ('The size & shape of the data to be resampled =', original.size,  '&', original.shape)

The size & shape of the template raster      = 353694 & (466, 759)
The size & shape of the data to be resampled = 8 & (2, 4)


Resample raster data

In [72]:
Resample = gis.MatchProjResNDV (source_file, target_fhsPCP, output_folder, resample = 'near', dtype = 'float32')

Check the size and shape of the raster files

In [74]:
## Check the size and shape of the resampled raster files
Resampled   = "/content/content/resampled/L1-PCP-D_resampled/PCPseason1_2022-10-01_to_2023-04-30.tif"
resampled   = gis.OpenAsArray(Resampled , nan_values=True)

print ('The size & shape of the resampled data =', resampled.size,  '&', resampled.shape)

The size & shape of the resampled data = 353694 & (466, 759)


##Zip and download resampled data 

In [ ]:
!zip -r /content/resampled.zip /content/content/resampled
from google.colab import files
files.download(r'/content/resampled.zip')
